In [1]:
from xgboost import XGBRegressor
import json
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
# read data
training_x = list()
with open ("../training_X_with_text_features.json",'r') as data:
    for d in data:
        training_x = json.loads(d)
        
# read data
training_y = list()
with open ("../training_Y_with_text_features.json",'r') as data:
    for d in data:
        training_y = json.loads(d)
        
training_x = np.array(training_x)
training_y = np.array(training_y)

print(training_x.shape)
print(training_y.shape)

(1237673, 20)
(1237673,)


In [21]:
# select 20% of the data to do CV to tune parameters
part_training_x = training_x[:round(0.2*(training_x.shape[0])),:]
part_training_y = training_y[:round(0.2*(training_x.shape[0]))]

In [11]:
# we do not do feature selection for xgboost
# because xgboost itself will find important/unimportant features
# we set threshold as a parameter to select features

In [31]:
# we use grid search to tune the hyperparameters
# hyper parameters list

max_depth_list = [2,4,6,8,10]
n_estimators_list = [*range(100, 600, 100)]
learning_rate_list = [0.05, 0.01]
colsample_bytree_list = [0.6,0.8,1]

In [34]:
results_dict = dict()
for a in max_depth_list:
    for b in n_estimators_list:
        for c in learning_rate_list:
            for d in colsample_bytree_list:
                
                xgboost = XGBRegressor(objective='reg:squarederror', n_jobs=-1, 
                                       reg_alpha = 0.5, reg_lambda=0.5, radom_state=2020,
                                       max_depth=a, n_estimators=b, learning_rate=c, colsample_bytree=d)

                cv_results = cross_validate(xgboost, part_training_x, part_training_y, cv=3, scoring=('neg_mean_squared_error'))
                results = -cv_results['test_score']
                key = "{},{},{},{}".format(a,b,c,d)
                results_dict[key] = round(np.mean(results),5)
                print(round(np.mean(results),3), "{},{},{},{}".format(a,b,c,d))

1.159 2,100,0.05,0.6
1.159 2,100,0.05,0.8
1.159 2,100,0.05,1
1.188 2,100,0.01,0.6
1.188 2,100,0.01,0.8
1.188 2,100,0.01,1
1.159 2,200,0.05,0.6
1.159 2,200,0.05,0.8
1.159 2,200,0.05,1
1.173 2,200,0.01,0.6
1.173 2,200,0.01,0.8
1.173 2,200,0.01,1
1.163 2,300,0.05,0.6
1.163 2,300,0.05,0.8
1.163 2,300,0.05,1
1.166 2,300,0.01,0.6
1.166 2,300,0.01,0.8
1.166 2,300,0.01,1
1.169 2,400,0.05,0.6
1.169 2,400,0.05,0.8
1.169 2,400,0.05,1
1.163 2,400,0.01,0.6
1.163 2,400,0.01,0.8
1.163 2,400,0.01,1
1.177 2,500,0.05,0.6
1.177 2,500,0.05,0.8
1.177 2,500,0.05,1
1.161 2,500,0.01,0.6
1.161 2,500,0.01,0.8
1.161 2,500,0.01,1
1.159 4,100,0.05,0.6
1.159 4,100,0.05,0.8
1.159 4,100,0.05,1
1.188 4,100,0.01,0.6
1.188 4,100,0.01,0.8
1.188 4,100,0.01,1
1.159 4,200,0.05,0.6
1.159 4,200,0.05,0.8
1.159 4,200,0.05,1
1.173 4,200,0.01,0.6
1.173 4,200,0.01,0.8
1.173 4,200,0.01,1
1.163 4,300,0.05,0.6
1.163 4,300,0.05,0.8
1.163 4,300,0.05,1
1.166 4,300,0.01,0.6
1.166 4,300,0.01,0.8
1.166 4,300,0.01,1
1.169 4,400,0.05,0.6
1.1

In [42]:
sorted_results = sorted(results_dict.items(),key = lambda x:x[0], reverse = False)

In [47]:
sorted_results[:10]

[(1.1587, '10,200,0.05,1'),
 (1.15901, '10,100,0.05,1'),
 (1.16089, '10,500,0.01,1'),
 (1.16271, '10,400,0.01,1'),
 (1.16318, '10,300,0.05,1'),
 (1.16607, '10,300,0.01,1'),
 (1.16939, '10,400,0.05,1'),
 (1.17293, '10,200,0.01,1'),
 (1.17687, '10,500,0.05,1'),
 (1.18777, '10,100,0.01,1')]

In [54]:
# it shows that max_depth should be = 10, colsample_bytree_list should be = 1
# learning rate seems does not matter
# tree number = 200 is locally optimial

In [56]:
# re-tune
max_depth_list = [8,10,12,14,20] # add 20 to see whether the deeper the tree is, the better the result is
n_estimators_list = [150,200,250,300,350,400]
learning_rate_list = [0.01,0.05] # 0.05 and 0.01 do not differ too much
colsample_bytree_list = [1] # should be 1

In [57]:
results_dict = dict()
for a in max_depth_list:
    for b in n_estimators_list:
        for c in learning_rate_list:
            for d in colsample_bytree_list:
                
                xgboost = XGBRegressor(objective='reg:squarederror', n_jobs=-1, 
                                       reg_alpha = 0.5, reg_lambda=0.5, radom_state=2020,
                                       max_depth=a, n_estimators=b, learning_rate=c, colsample_bytree=d)

                cv_results = cross_validate(xgboost, part_training_x, part_training_y, cv=3, scoring=('neg_mean_squared_error'))
                results = -cv_results['test_score']
                key = "{},{},{},{}".format(a,b,c,d)
                results_dict[key] = round(np.mean(results),5)
                print(round(np.mean(results),3), "{},{},{},{}".format(a,b,c,d))

1.179 8,150,0.01,1
1.158 8,150,0.05,1
1.173 8,200,0.01,1
1.159 8,200,0.05,1
1.169 8,250,0.01,1
1.161 8,250,0.05,1
1.166 8,300,0.01,1
1.163 8,300,0.05,1
1.164 8,350,0.01,1
1.166 8,350,0.05,1
1.163 8,400,0.01,1
1.169 8,400,0.05,1
1.179 10,150,0.01,1
1.158 10,150,0.05,1
1.173 10,200,0.01,1
1.159 10,200,0.05,1
1.169 10,250,0.01,1
1.161 10,250,0.05,1
1.166 10,300,0.01,1
1.163 10,300,0.05,1
1.164 10,350,0.01,1
1.166 10,350,0.05,1
1.163 10,400,0.01,1
1.169 10,400,0.05,1
1.179 12,150,0.01,1
1.158 12,150,0.05,1
1.173 12,200,0.01,1
1.159 12,200,0.05,1
1.169 12,250,0.01,1
1.161 12,250,0.05,1
1.166 12,300,0.01,1
1.163 12,300,0.05,1
1.164 12,350,0.01,1
1.166 12,350,0.05,1
1.163 12,400,0.01,1
1.169 12,400,0.05,1
1.179 14,150,0.01,1
1.158 14,150,0.05,1
1.173 14,200,0.01,1
1.159 14,200,0.05,1
1.169 14,250,0.01,1
1.161 14,250,0.05,1
1.166 14,300,0.01,1
1.163 14,300,0.05,1
1.164 14,350,0.01,1
1.166 14,350,0.05,1
1.163 14,400,0.01,1
1.169 14,400,0.05,1
1.179 20,150,0.01,1
1.158 20,150,0.05,1
1.173 20,200

In [59]:
sorted_results = sorted(results_dict.items(),key = lambda x:x[1], reverse = False)
sorted_results[:5]

[('8,150,0.05,1', 1.15795),
 ('10,150,0.05,1', 1.15795),
 ('12,150,0.05,1', 1.15795),
 ('14,150,0.05,1', 1.15795),
 ('20,150,0.05,1', 1.15795)]

In [3]:
# read testing data
testing_x = list()
with open ("../testing_X_with_text_features.json",'r') as data:
    for d in data:
        testing_x = json.loads(d)
        
# read data
testing_y = list()
with open ("../testing_Y_with_text_features.json",'r') as data:
    for d in data:
        testing_y = json.loads(d)
        
testing_x = np.array(testing_x)
testing_y = np.array(testing_y)

print(testing_x.shape)
print(testing_y.shape)

(280588, 20)
(280588,)


In [11]:
xgboost = XGBRegressor(objective='reg:squarederror', n_jobs=-1, 
                       reg_alpha = 0.5, reg_lambda=0.5, radom_state=2020,
                       max_depth=8, n_estimators=150, learning_rate=0.05, colsample_bytree=1)
xgboost.fit(training_x,training_y)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.05, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=150, n_jobs=-1, num_parallel_tree=1,
             objective='reg:squarederror', radom_state=2020, random_state=0,
             reg_alpha=0.5, reg_lambda=0.5, scale_pos_weight=1, subsample=1,
             tree_method=None, validate_parameters=False, verbosity=None)

In [12]:
predicted_ratings = xgboost.predict(testing_x)
predicted_ratings

array([3.9623332, 3.4999278, 3.7187412, ..., 1.7249293, 3.294361 ,
       3.7275891], dtype=float32)

In [13]:
rmse = sqrt(mean_squared_error(predicted_ratings, testing_y))

In [14]:
rmse

1.3052879027153284

In [16]:
feature_importances = dict()
for i in range(20):
    feature_importances[i] = xgboost.feature_importances_[i]

In [19]:
sorted(feature_importances.items(), key=lambda x:x[1], reverse=True)

[(5, 0.43537405),
 (1, 0.3339936),
 (7, 0.03962766),
 (14, 0.026047422),
 (0, 0.019585058),
 (9, 0.019291928),
 (19, 0.01710789),
 (2, 0.013922511),
 (6, 0.013712737),
 (11, 0.013301014),
 (16, 0.012709208),
 (17, 0.012649105),
 (12, 0.012256892),
 (4, 0.010883451),
 (10, 0.010047506),
 (15, 0.009489982),
 (3, 0.0),
 (8, 0.0),
 (13, 0.0),
 (18, 0.0)]

In [ ]:
# feature No. 5, 1, 7, 14, 0 are more important
# this can be compared with important features in other models